In [13]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [14]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/nul

In [15]:
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz

In [16]:
!ls

sample_data		   spark-3.1.1-bin-hadoop3.2.tgz
spark-3.1.1-bin-hadoop3.2  spark-3.1.1-bin-hadoop3.2.tgz.1


In [17]:
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [18]:
!pip install -q findspark

In [23]:
import os
os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [20]:
!pwd

/content


In [25]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Lab2").getOrCreate()
spark

In [28]:
ls

AirPollution.csv            spark-3.1.1-bin-hadoop3.2.tgz
sample_data/                spark-3.1.1-bin-hadoop3.2.tgz.1
spark-3.1.1-bin-hadoop3.2/


In [30]:
df = spark.read.csv("AirPollution.csv", sep=",", header=True)

In [31]:
df.show(5, truncate = False)

+---+----+-----+---+----+-----+----+----+----+----+-----+---+---+
|No |year|month|day|hour|pm2.5|DEWP|TEMP|PRES|cbwd|Iws  |Is |Ir |
+---+----+-----+---+----+-----+----+----+----+----+-----+---+---+
|1  |2010|1    |1  |0   |NA   |-21 |-11 |1021|NW  |1.79 |0  |0  |
|2  |2010|1    |1  |1   |NA   |-21 |-12 |1020|NW  |4.92 |0  |0  |
|3  |2010|1    |1  |2   |NA   |-21 |-11 |1019|NW  |6.71 |0  |0  |
|4  |2010|1    |1  |3   |NA   |-21 |-14 |1019|NW  |9.84 |0  |0  |
|5  |2010|1    |1  |4   |NA   |-20 |-12 |1018|NW  |12.97|0  |0  |
+---+----+-----+---+----+-----+----+----+----+----+-----+---+---+
only showing top 5 rows



In [32]:
df.columns

['No',
 'year',
 'month',
 'day',
 'hour',
 'pm2.5',
 'DEWP',
 'TEMP',
 'PRES',
 'cbwd',
 'Iws',
 'Is',
 'Ir']

In [33]:
df.dtypes

[('No', 'string'),
 ('year', 'string'),
 ('month', 'string'),
 ('day', 'string'),
 ('hour', 'string'),
 ('pm2.5', 'string'),
 ('DEWP', 'string'),
 ('TEMP', 'string'),
 ('PRES', 'string'),
 ('cbwd', 'string'),
 ('Iws', 'string'),
 ('Is', 'string'),
 ('Ir', 'string')]

In [35]:
from traitlets.traitlets import TraitError
df = spark.read.csv("AirPollution.csv", sep=",",  header=True, inferSchema=True)

In [36]:
df.dtypes

[('No', 'int'),
 ('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('hour', 'int'),
 ('pm2.5', 'string'),
 ('DEWP', 'int'),
 ('TEMP', 'double'),
 ('PRES', 'double'),
 ('cbwd', 'string'),
 ('Iws', 'double'),
 ('Is', 'int'),
 ('Ir', 'int')]

In [37]:
df.printSchema()

root
 |-- No: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- pm2.5: string (nullable = true)
 |-- DEWP: integer (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- PRES: double (nullable = true)
 |-- cbwd: string (nullable = true)
 |-- Iws: double (nullable = true)
 |-- Is: integer (nullable = true)
 |-- Ir: integer (nullable = true)



In [42]:
from pyspark.sql.types import *

lables = [
    ('No', IntegerType()),
    ('year', IntegerType()),
    ('month', IntegerType()),
    ('day', IntegerType()),
    ('hour', IntegerType()),
    ('pm2.5', StringType()),
    ('DEWP', IntegerType()),
    ('TEMP', DoubleType()),
    ('PRES', DoubleType()),
    ('cbwd', StringType()),
    ('Iws', DoubleType()),
    ('Is', IntegerType()),
    ('Ir', IntegerType())
]

In [43]:
schema = StructType([StructField(x[0], x[1]) for x in lables])
schema

StructType(List(StructField(No,IntegerType,true),StructField(year,IntegerType,true),StructField(month,IntegerType,true),StructField(day,IntegerType,true),StructField(hour,IntegerType,true),StructField(pm2.5,StringType,true),StructField(DEWP,IntegerType,true),StructField(TEMP,DoubleType,true),StructField(PRES,DoubleType,true),StructField(cbwd,StringType,true),StructField(Iws,DoubleType,true),StructField(Is,IntegerType,true),StructField(Ir,IntegerType,true)))

In [45]:
df = spark.read.csv('AirPollution.csv', header=True, sep=",", schema = schema, inferSchema = True)
df.show(5)


+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+
| No|year|month|day|hour|pm2.5|DEWP| TEMP|  PRES|cbwd|  Iws| Is| Ir|
+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+
|  1|2010|    1|  1|   0|   NA| -21|-11.0|1021.0|  NW| 1.79|  0|  0|
|  2|2010|    1|  1|   1|   NA| -21|-12.0|1020.0|  NW| 4.92|  0|  0|
|  3|2010|    1|  1|   2|   NA| -21|-11.0|1019.0|  NW| 6.71|  0|  0|
|  4|2010|    1|  1|   3|   NA| -21|-14.0|1019.0|  NW| 9.84|  0|  0|
|  5|2010|    1|  1|   4|   NA| -20|-12.0|1018.0|  NW|12.97|  0|  0|
+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+
only showing top 5 rows



In [47]:
df.select(df['year']).show(truncate=False)

+----+
|year|
+----+
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
|2010|
+----+
only showing top 20 rows



In [49]:
df.select(df.year, df.month, df.day, df.hour).show(truncate=False)

+----+-----+---+----+
|year|month|day|hour|
+----+-----+---+----+
|2010|1    |1  |0   |
|2010|1    |1  |1   |
|2010|1    |1  |2   |
|2010|1    |1  |3   |
|2010|1    |1  |4   |
|2010|1    |1  |5   |
|2010|1    |1  |6   |
|2010|1    |1  |7   |
|2010|1    |1  |8   |
|2010|1    |1  |9   |
|2010|1    |1  |10  |
|2010|1    |1  |11  |
|2010|1    |1  |12  |
|2010|1    |1  |13  |
|2010|1    |1  |14  |
|2010|1    |1  |15  |
|2010|1    |1  |16  |
|2010|1    |1  |17  |
|2010|1    |1  |18  |
|2010|1    |1  |19  |
+----+-----+---+----+
only showing top 20 rows



In [51]:
from pyspark.sql.functions import col
df.select(col('TEMP')).show(truncate=False)

+-----+
|TEMP |
+-----+
|-11.0|
|-12.0|
|-11.0|
|-14.0|
|-12.0|
|-10.0|
|-9.0 |
|-9.0 |
|-9.0 |
|-8.0 |
|-7.0 |
|-5.0 |
|-5.0 |
|-3.0 |
|-2.0 |
|-1.0 |
|-2.0 |
|-3.0 |
|-5.0 |
|-4.0 |
+-----+
only showing top 20 rows



In [52]:
from pyspark.sql.functions import lit,concat
df=df.withColumn('new_column',lit("New")).withColumn('second_new_column',concat(col('year'), lit(' '), col('month')))
df.show()

+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+----------+-----------------+
| No|year|month|day|hour|pm2.5|DEWP| TEMP|  PRES|cbwd|  Iws| Is| Ir|new_column|second_new_column|
+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+----------+-----------------+
|  1|2010|    1|  1|   0|   NA| -21|-11.0|1021.0|  NW| 1.79|  0|  0|       New|           2010 1|
|  2|2010|    1|  1|   1|   NA| -21|-12.0|1020.0|  NW| 4.92|  0|  0|       New|           2010 1|
|  3|2010|    1|  1|   2|   NA| -21|-11.0|1019.0|  NW| 6.71|  0|  0|       New|           2010 1|
|  4|2010|    1|  1|   3|   NA| -21|-14.0|1019.0|  NW| 9.84|  0|  0|       New|           2010 1|
|  5|2010|    1|  1|   4|   NA| -20|-12.0|1018.0|  NW|12.97|  0|  0|       New|           2010 1|
|  6|2010|    1|  1|   5|   NA| -19|-10.0|1017.0|  NW| 16.1|  0|  0|       New|           2010 1|
|  7|2010|    1|  1|   6|   NA| -19| -9.0|1017.0|  NW|19.23|  0|  0|       New|           2010 1|
|  8|2010|    1|  1|

In [53]:
df=df.withColumnRenamed('second_new_column','demo feature')
df.show()

+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+----------+------------+
| No|year|month|day|hour|pm2.5|DEWP| TEMP|  PRES|cbwd|  Iws| Is| Ir|new_column|demo feature|
+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+----------+------------+
|  1|2010|    1|  1|   0|   NA| -21|-11.0|1021.0|  NW| 1.79|  0|  0|       New|      2010 1|
|  2|2010|    1|  1|   1|   NA| -21|-12.0|1020.0|  NW| 4.92|  0|  0|       New|      2010 1|
|  3|2010|    1|  1|   2|   NA| -21|-11.0|1019.0|  NW| 6.71|  0|  0|       New|      2010 1|
|  4|2010|    1|  1|   3|   NA| -21|-14.0|1019.0|  NW| 9.84|  0|  0|       New|      2010 1|
|  5|2010|    1|  1|   4|   NA| -20|-12.0|1018.0|  NW|12.97|  0|  0|       New|      2010 1|
|  6|2010|    1|  1|   5|   NA| -19|-10.0|1017.0|  NW| 16.1|  0|  0|       New|      2010 1|
|  7|2010|    1|  1|   6|   NA| -19| -9.0|1017.0|  NW|19.23|  0|  0|       New|      2010 1|
|  8|2010|    1|  1|   7|   NA| -19| -9.0|1017.0|  NW|21.02|  0|  0|  

In [54]:
df.collect()

[Row(No=1, year=2010, month=1, day=1, hour=0, pm2.5='NA', DEWP=-21, TEMP=-11.0, PRES=1021.0, cbwd='NW', Iws=1.79, Is=0, Ir=0, new_column='New', demo feature='2010 1'),
 Row(No=2, year=2010, month=1, day=1, hour=1, pm2.5='NA', DEWP=-21, TEMP=-12.0, PRES=1020.0, cbwd='NW', Iws=4.92, Is=0, Ir=0, new_column='New', demo feature='2010 1'),
 Row(No=3, year=2010, month=1, day=1, hour=2, pm2.5='NA', DEWP=-21, TEMP=-11.0, PRES=1019.0, cbwd='NW', Iws=6.71, Is=0, Ir=0, new_column='New', demo feature='2010 1'),
 Row(No=4, year=2010, month=1, day=1, hour=3, pm2.5='NA', DEWP=-21, TEMP=-14.0, PRES=1019.0, cbwd='NW', Iws=9.84, Is=0, Ir=0, new_column='New', demo feature='2010 1'),
 Row(No=5, year=2010, month=1, day=1, hour=4, pm2.5='NA', DEWP=-20, TEMP=-12.0, PRES=1018.0, cbwd='NW', Iws=12.97, Is=0, Ir=0, new_column='New', demo feature='2010 1'),
 Row(No=6, year=2010, month=1, day=1, hour=5, pm2.5='NA', DEWP=-19, TEMP=-10.0, PRES=1017.0, cbwd='NW', Iws=16.1, Is=0, Ir=0, new_column='New', demo feature='2

In [55]:
df.groupBy('DEWP').count().show()

+----+-----+
|DEWP|count|
+----+-----+
| -35|    4|
| -13|  743|
| -33|   14|
| -20|  678|
| -21|  624|
| -17|  752|
|  -1|  856|
|  28|   11|
|  26|  110|
|  27|   36|
| -10|  864|
| -11|  896|
|  12| 1076|
| -27|  115|
|  22| 1145|
| -15|  789|
| -37|    3|
|   1|  699|
| -39|    1|
|  13| 1053|
+----+-----+
only showing top 20 rows



In [56]:
df.show()

+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+----------+------------+
| No|year|month|day|hour|pm2.5|DEWP| TEMP|  PRES|cbwd|  Iws| Is| Ir|new_column|demo feature|
+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+----------+------------+
|  1|2010|    1|  1|   0|   NA| -21|-11.0|1021.0|  NW| 1.79|  0|  0|       New|      2010 1|
|  2|2010|    1|  1|   1|   NA| -21|-12.0|1020.0|  NW| 4.92|  0|  0|       New|      2010 1|
|  3|2010|    1|  1|   2|   NA| -21|-11.0|1019.0|  NW| 6.71|  0|  0|       New|      2010 1|
|  4|2010|    1|  1|   3|   NA| -21|-14.0|1019.0|  NW| 9.84|  0|  0|       New|      2010 1|
|  5|2010|    1|  1|   4|   NA| -20|-12.0|1018.0|  NW|12.97|  0|  0|       New|      2010 1|
|  6|2010|    1|  1|   5|   NA| -19|-10.0|1017.0|  NW| 16.1|  0|  0|       New|      2010 1|
|  7|2010|    1|  1|   6|   NA| -19| -9.0|1017.0|  NW|19.23|  0|  0|       New|      2010 1|
|  8|2010|    1|  1|   7|   NA| -19| -9.0|1017.0|  NW|21.02|  0|  0|  

In [57]:
df.groupBy('DEWP','TEMP').count().show()

+----+-----+-----+
|DEWP| TEMP|count|
+----+-----+-----+
| -34|  3.0|    1|
| -24|-11.0|   15|
| -23|-15.0|    2|
| -18|-15.0|    8|
|  13| 19.0|   49|
| -23|-17.0|    1|
| -11|  1.0|   30|
|   3| 11.0|   28|
|  11| 30.0|   23|
|  -9|  2.0|   42|
|  -8| 10.0|   21|
|   0|  9.0|   25|
|  -3| 19.0|   35|
|  -2| 18.0|   22|
|  10| 25.0|   23|
| -11| 24.0|    2|
|  17| 19.0|  174|
|  -4| 23.0|   10|
|  14| 22.0|   50|
|  21| 29.0|   55|
+----+-----+-----+
only showing top 20 rows



In [59]:
df.drop('new_column').show()

+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+------------+
| No|year|month|day|hour|pm2.5|DEWP| TEMP|  PRES|cbwd|  Iws| Is| Ir|demo feature|
+---+----+-----+---+----+-----+----+-----+------+----+-----+---+---+------------+
|  1|2010|    1|  1|   0|   NA| -21|-11.0|1021.0|  NW| 1.79|  0|  0|      2010 1|
|  2|2010|    1|  1|   1|   NA| -21|-12.0|1020.0|  NW| 4.92|  0|  0|      2010 1|
|  3|2010|    1|  1|   2|   NA| -21|-11.0|1019.0|  NW| 6.71|  0|  0|      2010 1|
|  4|2010|    1|  1|   3|   NA| -21|-14.0|1019.0|  NW| 9.84|  0|  0|      2010 1|
|  5|2010|    1|  1|   4|   NA| -20|-12.0|1018.0|  NW|12.97|  0|  0|      2010 1|
|  6|2010|    1|  1|   5|   NA| -19|-10.0|1017.0|  NW| 16.1|  0|  0|      2010 1|
|  7|2010|    1|  1|   6|   NA| -19| -9.0|1017.0|  NW|19.23|  0|  0|      2010 1|
|  8|2010|    1|  1|   7|   NA| -19| -9.0|1017.0|  NW|21.02|  0|  0|      2010 1|
|  9|2010|    1|  1|   8|   NA| -19| -9.0|1017.0|  NW|24.15|  0|  0|      2010 1|
| 10|2010|    1|

In [61]:
total_count=df.count()
print("Total Count: " + str(total_count))

dewp = df.filter((col('DEWP')==-21) & (col('TEMP') <= 4)).count()
print("DEWP Count: " + str(dewp))

df.filter((col('DEWP')==-21) & (col('TEMP') <= 4 )).show()

Total Count: 43824
DEWP Count: 546
+---+----+-----+---+----+-----+----+-----+------+----+------+---+---+----------+------------+
| No|year|month|day|hour|pm2.5|DEWP| TEMP|  PRES|cbwd|   Iws| Is| Ir|new_column|demo feature|
+---+----+-----+---+----+-----+----+-----+------+----+------+---+---+----------+------------+
|  1|2010|    1|  1|   0|   NA| -21|-11.0|1021.0|  NW|  1.79|  0|  0|       New|      2010 1|
|  2|2010|    1|  1|   1|   NA| -21|-12.0|1020.0|  NW|  4.92|  0|  0|       New|      2010 1|
|  3|2010|    1|  1|   2|   NA| -21|-11.0|1019.0|  NW|  6.71|  0|  0|       New|      2010 1|
|  4|2010|    1|  1|   3|   NA| -21|-14.0|1019.0|  NW|  9.84|  0|  0|       New|      2010 1|
| 79|2010|    1|  4|   6|   20| -21|-12.0|1027.0|  NW| 64.81|  0|  0|       New|      2010 1|
| 80|2010|    1|  4|   7|   29| -21|-13.0|1027.0|  NW| 73.75|  0|  0|       New|      2010 1|
| 85|2010|    1|  4|  12|   29| -21|-11.0|1030.0|  NW|117.55|  0|  0|       New|      2010 1|
| 87|2010|    1|  4|  14|

In [62]:
df.select('DEWP').distinct().show()

+----+
|DEWP|
+----+
| -35|
| -13|
| -33|
| -20|
| -21|
| -17|
|  -1|
|  28|
|  26|
|  27|
| -10|
| -11|
|  12|
| -27|
|  22|
| -15|
| -37|
|   1|
| -39|
|  13|
+----+
only showing top 20 rows



In [67]:
df.select('DEWP','TEMP').distinct().show()

+----+-----+
|DEWP| TEMP|
+----+-----+
| -24|-11.0|
| -23|-15.0|
| -18|-15.0|
|  13| 19.0|
| -23|-17.0|
| -34|  3.0|
| -11|  1.0|
|   3| 11.0|
|  11| 30.0|
|  -9|  2.0|
|  -8| 10.0|
|   0|  9.0|
|  -3| 19.0|
|  -2| 18.0|
|  10| 25.0|
| -11| 24.0|
|  17| 19.0|
|  -4| 23.0|
|  14| 22.0|
|  21| 29.0|
+----+-----+
only showing top 20 rows



In [68]:
df.select('DEWP','TEMP').distinct().count()

1741